In [29]:
#lib
import numpy as np
import tensorflow as tf
from tensorflow import keras

In [17]:
#tensorflow dataset
import tensorflow_datasets as tfds

In [18]:
mnist_data,mnist_info = tfds.load(name='mnist',with_info=True, as_supervised=True)

In [19]:
mnist_info

tfds.core.DatasetInfo(
    name='mnist',
    version=3.0.1,
    description='The MNIST database of handwritten digits.',
    homepage='http://yann.lecun.com/exdb/mnist/',
    features=FeaturesDict({
        'image': Image(shape=(28, 28, 1), dtype=tf.uint8),
        'label': ClassLabel(shape=(), dtype=tf.int64, num_classes=10),
    }),
    total_num_examples=70000,
    splits={
        'test': 10000,
        'train': 60000,
    },
    supervised_keys=('image', 'label'),
    citation="""@article{lecun2010mnist,
      title={MNIST handwritten digit database},
      author={LeCun, Yann and Cortes, Corinna and Burges, CJ},
      journal={ATT Labs [Online]. Available: http://yann.lecun.com/exdb/mnist},
      volume={2},
      year={2010}
    }""",
    redistribution_info=,
)

In [20]:
mnist_train,mnist_test = mnist_data['train'],mnist_data['test']


In [21]:

#valid dataset
percentage_number_for_valid = 0.1 * mnist_info.splits['train'].num_examples
percentage_number_for_valid = tf.cast(percentage_number_for_valid,tf.int64)

#test purpose
number_test_samples = 0.1 * mnist_info.splits['test'].num_examples
number_test_samples = tf.cast(number_test_samples,tf.int64)

In [22]:
percentage_number_for_valid, number_test_samples

(<tf.Tensor: shape=(), dtype=int64, numpy=6000>,
 <tf.Tensor: shape=(), dtype=int64, numpy=1000>)

In [23]:
#creating a scaler
def scale(image,label):
  image = tf.cast(image,tf.float32)
  image = image/ 255
  return image,label

In [24]:
scaled_tain_and_valid_data = mnist_train.map(scale)
test_data = mnist_test.map(scale)

In [25]:
#shuffle data
BUFFER_SIZE = 10000

shuffle_train_and_valid = scaled_tain_and_valid_data.shuffle(BUFFER_SIZE)

#creating new valid and train data
validation_data = shuffle_train_and_valid.take(percentage_number_for_valid)
train_data = shuffle_train_and_valid.skip(percentage_number_for_valid)


In [26]:
#BATCH
BATCH_SIZE = 100
train_data = train_data.batch(BATCH_SIZE)
validation_data = validation_data.batch(percentage_number_for_valid)
test_data = test_data.batch(number_test_samples)

In [27]:
validation_input, validation_traget = next(iter(validation_data))

In [34]:
### MODEL ###
inpute_size = 784
output_size = 10
hidden_layer_size = 150

model = tf.keras.Sequential([
                             keras.layers.Flatten(input_shape = (28,28,1)),
                             keras.layers.Dense(hidden_layer_size, activation='relu'),
                             keras.layers.Dense(hidden_layer_size,activation='relu'),
                             keras.layers.Dense(output_size, activation='softmax')
])

In [35]:
### OPTIMIZER ###
model.compile(optimizer='adam',loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [36]:
### TRAINING ###
NUM_EPOCHS = 5

model.fit(train_data, epochs=NUM_EPOCHS, validation_data=(validation_input,validation_traget),verbose = 2)

Epoch 1/5
540/540 - 3s - loss: 0.2928 - accuracy: 0.9158 - val_loss: 0.1612 - val_accuracy: 0.9547
Epoch 2/5
540/540 - 3s - loss: 0.1186 - accuracy: 0.9641 - val_loss: 0.0963 - val_accuracy: 0.9717
Epoch 3/5
540/540 - 3s - loss: 0.0806 - accuracy: 0.9755 - val_loss: 0.0776 - val_accuracy: 0.9788
Epoch 4/5
540/540 - 3s - loss: 0.0632 - accuracy: 0.9797 - val_loss: 0.0640 - val_accuracy: 0.9807
Epoch 5/5
540/540 - 3s - loss: 0.0469 - accuracy: 0.9854 - val_loss: 0.0497 - val_accuracy: 0.9857


In [37]:
### TEST ###
test_loss, test_accuracy = model.evaluate(test_data)

10/10 [==============================] - 1s 102ms/step - loss: 0.0742 - accuracy: 0.9760


In [38]:
test_loss,test_accuracy

(0.0742301419377327, 0.9760000109672546)

In [40]:
print(f'Test Loss: {test_loss:.2f}% \n Test Accuracy: {test_accuracy:.2f}%')

Test Loss: 0.07% 
 Test Accuracy: 0.98%
